In [34]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [46]:
#config file 불러오기
# cfg = Config.fromfile('/opt/ml/CustomPipeline/baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_1x_coco.py')
cfg = Config.fromfile('/opt/ml/CustomPipeline/baseline/mmdetection/configs/detr/detr_r50_8x2_150e_coco.py')

classes = ("General trash","Paper","Papaer pack","Metal","Glass",
           "Plastic","Styrofoam","Plastic bag","Battery","Clothing")

#dataset 바꾸기
root = "/opt/ml/detection/dataset/"

cfg.data.test.classes = classes
cfg.data.test.img_prefix  = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize
cfg.data.test_mode=True

cfg.data.samples_per_gpu = 4

cfg.seed= 42
cfg.gpu_ids = [1]
cfg.work_dir = '/opt/ml/CustomPipeline/MyModel/work_dirs/detr_trash'

# cfg.model.roi_head.bbox_head.num_classes = 11
cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35,norm_type=2)
cfg.checkpoint_config = None

In [57]:
# build dataset & dataloader
print(cfg.data.test)
# print(cfg.data.train)
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

{'type': 'CocoDataset', 'ann_file': '/opt/ml/detection/dataset/test.json', 'img_prefix': '/opt/ml/detection/dataset/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1024, 1024), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 1}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'classes': ('General trash', 'Paper', 'Papaer pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')}
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [61]:
epoch = 'latest'
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

AssertionError: DETR: DETRHead: Expected class_weight to have type float. Found <class 'torch.Tensor'>.

In [50]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

completed: 0, elapsed: 0s

In [42]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
